# Scaling Test-Time Compute for Longer Thinking in LLMs

This notebook is originally based on a huggingface [cookbook](https://github.com/huggingface/cookbook/blob/main/notebooks/en/search_and_learn.ipynb).

❗️This notebook can only perform inference but not benchmark a searching strategy.

This needs T4 GPU and is only tested with `config.n=4`. `config.n=8` causes OOM. \
Execution Time of each query with these settings:
- best_of_n: 1 mins
- beam_search: 5 mins
- dvts: 2 mins

❗️Huge amount of time is needed for `pip install` and downloading models in every new RUNTIME. If any error occurs, restart the SESSION.

## 1. Install Dependencies

Since Colab comes with many pre-installed packages, leading to difficult-to-resolve version conflicts, we install dependencies in a local virtual environment and freeze them here.

In [ ]:
%%bash
echo "
accelerate==1.5.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.14
aiosignal==1.3.2
annotated-types==0.7.0
antlr4-python3-runtime==4.7.2
anyio==4.9.0
attrs==25.3.0
certifi==2025.1.31
charset-normalizer==3.4.1
click==8.1.8
cloudpickle==3.1.1
datasets==3.5.0
dill==0.3.8
diskcache==5.6.3
distro==1.9.0
einops==0.8.1
fastapi==0.115.12
filelock==3.18.0
frozenlist==1.5.0
fsspec==2024.12.0
gguf==0.10.0
h11==0.14.0
hf_transfer==0.1.9
httpcore==1.0.7
httptools==0.6.4
httpx==0.28.1
huggingface-hub==0.29.3
idna==3.10
importlib_metadata==8.6.1
iniconfig==2.1.0
interegular==0.3.3
isort==6.0.1
Jinja2==3.1.6
jiter==0.9.0
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
lark==1.2.2
latex2sympy2==1.9.1
llvmlite==0.44.0
lm-format-enforcer==0.10.6
MarkupSafe==3.0.2
mistral_common==1.5.4
mpmath==1.3.0
msgpack==1.1.0
msgspec==0.19.0
multidict==6.2.0
multiprocess==0.70.16
nest-asyncio==1.6.0
networkx==3.4.2
numba==0.61.0
numpy==1.26.4
nvidia-ml-py==12.570.86
openai==1.69.0
opencv-python-headless==4.11.0.86
outlines==0.0.46
packaging==24.2
pandas==2.2.3
partial-json-parser==0.2.1.1.post5
Pebble==5.1.1
pillow==11.1.0
pluggy==1.5.0
prometheus-fastapi-instrumentator==7.1.0
prometheus_client==0.21.1
propcache==0.3.1
protobuf==6.30.2
psutil==7.0.0
py-cpuinfo==9.0.0
pyairports==2.1.1
pyarrow==19.0.1
pycountry==24.6.1
pydantic==2.11.1
pydantic_core==2.33.0
pytest==8.3.5
python-dateutil==2.9.0.post0
python-dotenv==1.1.0
pytz==2025.2
PyYAML==6.0.2
pyzmq==26.3.0
ray==2.44.1
referencing==0.36.2
regex==2024.11.6
requests==2.32.3
rpds-py==0.24.0
ruff==0.11.2
safetensors==0.5.3
sentencepiece==0.2.0
six==1.17.0
sniffio==1.3.1
starlette==0.46.1
sympy==1.13.3
tiktoken==0.9.0
tokenizers==0.21.1
torch==2.4.0
torchvision==0.19.0
tqdm==4.67.1
transformers==4.50.3
typing-inspection==0.4.0
typing_extensions==4.13.0
tzdata==2025.2
urllib3==2.3.0
uvicorn==0.34.0
uvloop==0.21.0
vllm==0.6.3
watchfiles==1.0.4
websockets==15.0.1
word2number==1.1
xxhash==3.5.0
yarl==1.18.3
zipp==3.21.0
" > requirements.txt

❗️This ends with multiple errors; just ignore them, as we are not using those packages.

In [ ]:
!pip install -r requirements.txt

In [ ]:
!git clone https://github.com/choyerhuang/CSCI544-Project

❗️If `ImportError: No module named sal`, restart session and start again from here.

In [ ]:
%cd search-and-learn
!pip install -e '.[dev]'

/content/search-and-learn
Obtaining file:///content/search-and-learn
  Installing build dependencies ... done
  Checking if build backend supports build_editable ... done
  Getting requirements to build editable ... done
  Preparing editable metadata (pyproject.toml) ... done
  Building editable for search-and-learn (pyproject.toml) ... done
  Created wheel for search-and-learn: filename=search_and_learn-0.1.0-0.editable-py3-none-any.whl size=8677 sha256=fe7cb7fe333c08dabb4057bafc78ce4b28b51edc9940743c2a2a8c077827ba99
  Stored in directory: /tmp/pip-ephem-wheel-cache-q1u4vuaq/wheels/87/cf/59/86a78cb0faa390ac87785f5d31799a5587b8e52dc5308682f2
Successfully built search-and-learn
  Attempting uninstall: search-and-learn
    Found existing installation: search-and-learn 0.1.0
    Uninstalling search-and-learn-0.1.0:
      Successfully uninstalled search-and-learn-0.1.0


Log in to Hugging Face to access [meta-llama/Llama-3.2-1B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct), as it is a gated model! 🗝️  
If you haven't previously requested access, you'll need to submit a request before proceeding.

⚠️ Use your USC email to register an account. When requesting access, enter "University of Southern California" as your affiliation and select "Research Graduate"; otherwise, your request will be rejected.

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

## 2. Setup the Large Language Model (LLM) and the Process Reward Model (PRM) 💬

As illustrated in the diagram, the system consists of an LLM that generates intermediate answers based on user input, a [PRM model](https://huggingface.co/papers/2211.14275) that evaluates and scores these answers, and a search strategy that uses the PRM feedback to guide the subsequent steps in the search process until reaching the final answer.

Let’s begin by initializing each model. For the LLM, we’ll use the [meta-llama/Llama-3.2-1B-Instruct](https://huggingface.co/meta-llama/Llama-3.2-1B-Instruct) model, and for the PRM, we’ll use the [RLHFlow/Llama3.1-8B-PRM-Deepseek-Data](https://huggingface.co/RLHFlow/Llama3.1-8B-PRM-Deepseek-Data) model.




![system](https://huggingface.co/datasets/HuggingFaceH4/blogpost-images/resolve/main/system.png)

⬇️ Start again from here after **Restart session**.

In [ ]:
import torch
from vllm import LLM
from sal.models.reward_models import RLHFFlow

model_path="meta-llama/Llama-3.2-1B-Instruct"
prm_path="RLHFlow/Llama3.1-8B-PRM-Deepseek-Data"

llm = LLM(
    model=model_path,
    gpu_memory_utilization=0.5,  # Utilize 50% of GPU memory
    enable_prefix_caching=True,  # Optimize repeated prefix computations
    seed=42,                     # Set seed for reproducibility
    dtype='half',
    max_model_len=8192,
)

prm = RLHFFlow(prm_path)

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:211: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_fwd")
/usr/local/lib/python3.11/dist-packages/xformers/ops/fmha/flash.py:344: FutureWarning: `torch.library.impl_abstract` was renamed to `torch.library.register_fake`. Please use that instead; we will remove `torch.library.impl_abstract` in a future version of PyTorch.
  @torch.library.impl_abstract("xformers_flash::flash_bwd")


INFO 03-31 01:21:45 model_runner.py:1060] Starting to load model meta-llama/Llama-3.2-1B-Instruct...
INFO 03-31 01:21:46 selector.py:224] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 03-31 01:21:46 selector.py:115] Using XFormers backend.
INFO 03-31 01:21:46 weight_utils.py:243] Using model weights format ['*.safetensors']
INFO 03-31 01:21:46 weight_utils.py:288] No model.safetensors.index.json found in remote.


Loading safetensors checkpoint shards:   0% Completed | 0/1 [00:00<?, ?it/s]


INFO 03-31 01:22:00 model_runner.py:1071] Loading model weights took 2.3185 GB
INFO 03-31 01:22:02 gpu_executor.py:122] # GPU blocks: 6055, # CPU blocks: 8192
INFO 03-31 01:22:02 gpu_executor.py:126] Maximum concurrency for 8192 tokens per request: 11.83x
INFO 03-31 01:22:05 model_runner.py:1402] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 03-31 01:22:05 model_runner.py:1406] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 03-31 01:22:33 model_runner.py:1530] Graph capturing finished in 28 secs.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

### 2.1 Instantiate the Question, Search Strategy, and Call the Pipeline

Now that we've set up the LLM and PRM, let's proceed by defining the question, selecting a search strategy to retrieve relevant information, and calling the pipeline to process the question through the models.

1. **Instantiate the Question**: In this step, we define the input question that the system will answer, considering the given context.

2. **Search Strategy**: The system currently supports the following search strategies: `best_of_n`, `beam_search`, and `dvts` (see diagram). For this example, we'll use `best_of_n`, but you can easily switch to any of the other strategies based on your needs. We need to define some configuration parameters for the configuration of the search strategy. You can check the full list [here](https://github.com/huggingface/search-and-learn/blob/main/src/sal/config.py).

3. **Call the Pipeline**: With the question and search strategy in place, we’ll call the inference pipeline, processing the inputs through both the LLM and PRM to generate the final answer.

![](https://huggingface.co/datasets/HuggingFaceH4/blogpost-images/resolve/main/search-strategies.png)

The first step is to clearly define the question that the system will answer. This ensures that we have a precise task for the model to tackle.

In [ ]:
question_text = 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$'
input_batch = {"problem": [question_text]}

Next, we define the configuration, including parameters like the number of candidate answers `(N)`, and choose the search strategy that will be used. The search strategy dictates how we explore the potential answers. In this case, we'll use `best_of_n`.

With the question and configuration in place, we use the selected search strategy to generate multiple candidate answers. These candidates are evaluated based on their relevance and quality and the final answer is returned.


In [ ]:
from sal.config import Config
import os
os.chdir('/content/search-and-learn/')
from sal.search import beam_search, best_of_n, dvts, run_dynamic_beam_search, beam_search_ev, greedy_backtrack_search

config = Config()
config.n=4 # Number of answers to generate during the search
config.prm_batch_size=1
config.search_batch_size=1

search_result = best_of_n(x=input_batch, config=config, llm=llm, prm=prm)

### 2.2 Display the Final Result

Once the pipeline has processed the question through the LLM and PRM, we can display the final result. This result will be the model's output after considering the intermediate answers and scoring them using the PRM.

Here's how to display the final answer:

In [ ]:
search_result['pred'][0]

'## Step 1:  The given point in rectangular coordinates is $(0,3)$, which can be represented as $(x, y)$ where $x = 0$ and $y = 3$.\n## Step 2:  To convert this point from rectangular coordinates to polar coordinates, we need to find the radius $r$ and the angle $\\theta$.\n## Step 3:  The radius $r$ can be calculated using the formula $r = \\sqrt{x^2 + y^2}$.\n## Step 4:  Given that $x = 0$ and $y = 3$, we can substitute these values into the formula to get $r = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3$.\n## Step 5:  The angle $\\theta$ can be found using the arctangent function, $\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right)$.\n## Step 6:  Since $x = 0$, the angle $\\theta$ will be $\\frac{\\pi}{2}$, as the point lies on the positive y-axis.\n## Step 7:  Therefore, the polar coordinates of the point $(0,3)$ are $(3, \\frac{\\pi}{2})$.\n\nThe final answer is: $\\boxed{\\left(3, \\frac{\\pi}{2}\\right)}$'

The model’s output might include special tokens, such as `<|start_header_id|>` or `<|end_header_id|>`. To make the answer more readable, we can safely remove them before displaying it to the end user.

In [ ]:
formatted_output = search_result['pred'][0].replace("<|start_header_id|>assistant<|end_header_id|>\n\n", "").strip()
formatted_output

'## Step 1:  The given point in rectangular coordinates is $(0,3)$, which can be represented as $(x, y)$ where $x = 0$ and $y = 3$.\n## Step 2:  To convert this point from rectangular coordinates to polar coordinates, we need to find the radius $r$ and the angle $\\theta$.\n## Step 3:  The radius $r$ can be calculated using the formula $r = \\sqrt{x^2 + y^2}$.\n## Step 4:  Given that $x = 0$ and $y = 3$, we can substitute these values into the formula to get $r = \\sqrt{0^2 + 3^2} = \\sqrt{9} = 3$.\n## Step 5:  The angle $\\theta$ can be found using the arctangent function, $\\theta = \\tan^{-1}\\left(\\frac{y}{x}\\right)$.\n## Step 6:  Since $x = 0$, the angle $\\theta$ will be $\\frac{\\pi}{2}$, as the point lies on the positive y-axis.\n## Step 7:  Therefore, the polar coordinates of the point $(0,3)$ are $(3, \\frac{\\pi}{2})$.\n\nThe final answer is: $\\boxed{\\left(3, \\frac{\\pi}{2}\\right)}$'

After removing any special tokens, we can display the final answer to the user. Since the answer is based on markdown, it can be rendered properly by displaying it as markdown.

In [ ]:
from IPython.display import display, Markdown

display(Markdown(formatted_output))

## Step 1:  The given point in rectangular coordinates is $(0,3)$, which can be represented as $(x, y)$ where $x = 0$ and $y = 3$.
## Step 2:  To convert this point from rectangular coordinates to polar coordinates, we need to find the radius $r$ and the angle $\theta$.
## Step 3:  The radius $r$ can be calculated using the formula $r = \sqrt{x^2 + y^2}$.
## Step 4:  Given that $x = 0$ and $y = 3$, we can substitute these values into the formula to get $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.
## Step 5:  The angle $\theta$ can be found using the arctangent function, $\theta = \tan^{-1}\left(\frac{y}{x}\right)$.
## Step 6:  Since $x = 0$, the angle $\theta$ will be $\frac{\pi}{2}$, as the point lies on the positive y-axis.
## Step 7:  Therefore, the polar coordinates of the point $(0,3)$ are $(3, \frac{\pi}{2})$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

## 3. Assembling It All! 🧑‍🏭️

Now, let's create a method that encapsulates the entire pipeline. This will allow us to easily reuse the process in future applications, making it efficient and modular.

By combining the LLM, PRM, search strategy, and result display, we can simplify the workflow and ensure that it’s reusable for other tasks or questions.

We simplify the workflow, ensuring that it’s reusable for different tasks or questions. Additionally, we’ll track the time spent on each method so that we can **understand the practical implications** of using each strategy and configuration.

Here’s how we can structure the method:

In [ ]:
import time

def generate_with_search_and_learn(question, config, llm, prm, method='best_of_n'):
    """
    Generate an answer for a given question using the search-and-learn pipeline.

    Args:
    - question (str): The input question to generate an answer for.
    - config (Config): Configuration object containing parameters for search strategy.
    - llm (LLM): Pretrained large language model used for generating answers.
    - prm (RLHFFlow): Process reward model used for evaluating answers.
    - method (str): Search strategy to use. Options are 'best_of_n', 'beam_search', 'dvts'. Default is 'best_of_n'.

    Returns:
    - str: The formatted output after processing the question.
    """
    batch = {"problem": [question]}

    start_time = time.time()
    if method == 'best_of_n':
      result = best_of_n(x=batch, config=config, llm=llm, prm=prm)
    elif method == 'beam_search':
      result = beam_search(examples=batch, config=config, llm=llm, prm=prm)
    elif method == 'dvts':
      result = dvts(examples=batch, config=config, llm=llm, prm=prm)
    elif method == 'dynamic_beam':
      result = run_dynamic_beam_search(example_batch=batch, config=config, llm=llm, prm=prm)
    elif method == 'beam_search_ev':
      result = beam_search_ev(examples=batch, config=config, llm=llm, prm=prm)
    elif method == 'greedy_backtrack_search':
      result = greedy_backtrack_search(examples=batch, config=config, llm=llm, prm=prm)

    elapsed_time = time.time() - start_time
    print(f"\nFinished in {elapsed_time:.2f} seconds\n")

    tokenizer = llm.get_tokenizer()
    total_tokens = 0
    for completion in result['completions']:
        for comp in  completion:
            output_tokens = tokenizer.encode(comp)
            total_tokens += len(output_tokens)

    print(f"Total tokens in all completions: {total_tokens}")

    formatted_output = result['pred'][0].replace("<|start_header_id|>assistant<|end_header_id|>\n\n", "").strip()
    return formatted_output

### ⏳  3.1 Comparing Thinking Time for Each Strategy

Let’s compare the **thinking time** of three methods: `best_of_n`, `beam_search`, and `dvts`. Each method is evaluated using the same number of answers during the search process, measuring the time spent thinking in seconds and the number of generated tokens.

In the results below, the `best_of_n` method shows the least thinking time, while the `dvts` method takes the most time. However, `best_of_n` generates more tokens due to its simpler search strategy.

| **Method**      | **Number of Answers During Search** | **Thinking Time (Seconds)** | **Generated Tokens** |
|------------------|-------------------------------------|-----------------------------|-----------------------|
| **best_of_n**    | 8                                   | 3.54                        | 3087                  |
| **beam_search**  | 8                                   | 10.06                       | 2049                  |
| **dvts**         | 8                                   | 8.46                        | 2544                  |

This comparison illustrates the trade-offs between the strategies, balancing time spent thinking and the complexity of the search process.


#### 1. **Best of n**

We’ll begin by using the `best_of_n` strategy. Here’s how to track the thinking time for this method:

In [ ]:
question = 'Convert the point $(0,3)$ in rectangular coordinates to polar coordinates.  Enter your answer in the form $(r,\theta),$ where $r > 0$ and $0 \le \theta < 2 \pi.$'

config.n=4

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='best_of_n')


Finished in 61.41 seconds

Total tokens in all completions: 1034


In [ ]:
display(Markdown(formatted_output))

## Step 1:  To convert the point $(0, 3)$ from rectangular coordinates to polar coordinates, we need to calculate the radius $r$ and the angle $\theta$.
## Step 2:  The radius $r$ can be found using the formula $r = \sqrt{x^2 + y^2}$. In this case, $x = 0$ and $y = 3$, so $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.
## Step 3:  To find the angle $\theta$, we use the relationship between rectangular and polar coordinates: $\theta = \tan^{-1}\left(\frac{y}{x}\right)$. Since $x = 0$, this simplifies to $\theta = \tan^{-1}(0) = \frac{\pi}{2}$.
## Step 4:  Therefore, the polar coordinates of the point $(0, 3)$ are $\left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

#### 2. **Beam Search**

Now, let's try using the `beam_search` strategy.

In [ ]:
config.n=4
# beam search specific
config.sort_completed=True
config.filter_duplicates=True

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='beam_search')

Beam search iterations:  12%|█▎        | 5/40 [05:25<37:56, 65.03s/it]


Finished in 325.18 seconds

Total tokens in all completions: 1080


In [ ]:
display(Markdown(formatted_output))

## Step 1: To convert from rectangular coordinates (x, y) to polar coordinates (r, heta), we use the formulas $r = \sqrt{x^2 + y^2}$ and $heta = \tan^{-1}\left(\frac{y}{x}\right)$.

## Step 2: First, we substitute the given values (x, y) = (0, 3) into the formula for r: $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.

## Step 3: Next, we substitute the given values (x, y) = (0, 3) into the formula for heta: $heta = \tan^{-1}\left(\frac{3}{0}\right)$.

## Step 4: Since division by zero is undefined, we recognize that the point (0, 3) is on the positive y-axis, which implies that $heta = \frac{\pi}{2}$.

## Step 5: Thus, the polar coordinates of (0, 3) are $\left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

#### 2. **Beam Search with ensemble voting**

I have additional implement beam search with ensemble voting stategy as independent function.

In [ ]:
config.n=4
# beam search specific
config.approach = 'beam_search_ev'
config.sort_completed=True
config.filter_duplicates=True

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='beam_search_ev')

In [ ]:
display(Markdown(formatted_output))

#### 3. **Diverse Verifier Tree Search (DVTS)**

Let's try the `dvts` strategy.

In [ ]:
config.n=4
# dvts specific
config.n_beams = config.n // config.beam_width

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='dvts')

Beam search iterations:   5%|▌         | 2/40 [01:51<35:24, 55.91s/it]


Finished in 111.81 seconds

Total tokens in all completions: 988


In [ ]:
display(Markdown(formatted_output))

## Step 1:  To convert the point $(0,3)$ from rectangular coordinates to polar coordinates, we need to find the radius $r$ and the angle $heta$.
## Step 2:  The radius $r$ can be found using the formula $r = \sqrt{x^2 + y^2}$. In this case, $x = 0$ and $y = 3$, so $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.
## Step 3:  Next, we need to find the angle $heta$. The angle $heta$ can be found using the formula $heta = \arctan\left(\frac{y}{x}\right)$. However, since $x = 0$, the value of $heta$ will be $\frac{\pi}{2}$.
## Step 4:  So, we have found the polar coordinates $(r, heta)$ to be $(3, \frac{\pi}{2})$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

#### 4. **Dynamic Beam Search Method**

Let's try our new `Dynamic Beam search` strategy.

In [ ]:
# setting basic parameter
config.n = 4

# dynamic beam search parameters
config.approach = "dynamic_beam"
config.sort_completed = True
config.filter_duplicates = True
config.num_iterations = 10
config.dynamic_beam_delta = 0.3   # Beam score margin
config.min_beams = 2
config.max_beams = 4

formatted_output = generate_with_search_and_learn(
    question=question,
    config=config,
    llm=llm,
    prm=prm,
    method="dynamic_beam"
)

Dynamic Beam Search Steps:  70%|███████   | 7/10 [07:29<03:12, 64.28s/it]


Finished in 449.99 seconds

Total tokens in all completions: 1751


In [ ]:
display(Markdown(formatted_output))

## Step 1:  To convert the point $(0,3)$ from rectangular coordinates to polar coordinates, we'll use the formulas $r = \sqrt{x^2 + y^2}$ for the radial coordinate and $\theta = \tan^{-1}\left(\frac{y}{x}\right)$ for the angular coordinate.
## Step 2:  Given the point $(0,3)$, we can calculate the radial coordinate $r$ by substituting $x = 0$ and $y = 3$ into the formula $r = \sqrt{x^2 + y^2}$. This gives us $r = \sqrt{0^2 + 3^2} = \sqrt{9} = 3$.
## Step 3:  Next, we'll find the angular coordinate $\theta$ using the formula $\theta = \tan^{-1}\left(\frac{y}{x}\right)$. However, we notice that the point $(0,3)$ is on the positive y-axis, which means $\theta = \frac{\pi}{2}$.
## Step 4:  Therefore, after applying the formulas, we can express the rectangular coordinates $(0,3)$ in polar coordinates as $\left(3, \frac{\pi}{2}\right)$.

The final answer is: $\boxed{\left(3, \frac{\pi}{2}\right)}$

#### 4. **Greedy Backtrack Search Method**

Let's try our new `Greedy Backtrack Search` strategy.

In [ ]:
# setting basic parameter
config.n = 4

# dynamic beam search parameters
config.approach = "greedy_backtrack"
config.sort_completed = True
config.filter_duplicates = True
config.num_iterations = 10
config.max_backtrack_depth = 2         # (NEW) Optional: maximum levels to look back
config.early_stop_when_x_finished = 1

formatted_output = generate_with_search_and_learn(
    question=question,
    config=config,
    llm=llm,
    prm=prm,
    method="greedy_backtrack"
)

In [ ]:
display(Markdown(formatted_output))

### 🙋 3.2 Testing the System with a Simple Question

In this final example, we’ll test the system using a straightforward question to observe how it performs in simpler cases. This allows us to verify that the system works as expected even for basic queries.

Let's try the following question:

In [ ]:
question = 'What\'s the capital of Spain?'

config.n=4

formatted_output = generate_with_search_and_learn(question=question, config=config, llm=llm, prm=prm, method='best_of_n')


Finished in 45.39 seconds

Total tokens in all completions: 32


In [ ]:
display(Markdown(formatted_output))

The capital of Spain is Madrid.

Even though we set a larger number of candidate answers (`N`), the time spent thinking remains relatively small (1.03 seconds and 544 generated tokens). This demonstrates the system’s ability to efficiently handle easier problems, spending less time on them, while leveraging its enhanced capabilities for more complex questions.

🏆 **We now have a fully operational pipeline** that leverages test-time compute, enabling the system to "think longer" for more complicated queries, while also maintaining fast response times for straightforward questions.

This approach ensures the system can scale its thinking time based on the task's complexity, offering an efficient and responsive solution for both simple and challenging problems.


## 4. Continuing the Journey and Resources 🧑‍🎓️

If you're eager to continue exploring, be sure to check out the original experimental [blog](https://huggingface.co/spaces/HuggingFaceH4/blogpost-scaling-test-time-compute) and all the references mentioned within it. These resources will deepen your understanding of test-time compute, its benefits, and its applications in LLMs.


Happy learning and experimenting! 🚀